<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import numpy as np 
import pandas as pd 
import os as os 

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.
#os.mkdir('/users/dmarm/downloads/excellab')
# Create a files variable that contains all of our data files.

In [3]:
files = os.listdir('data')

In [4]:
files

['Jan 1.xlsx',
 'Jan 10.xlsx',
 'Jan 11.xlsx',
 'Jan 12.xlsx',
 'Jan 13.xlsx',
 'Jan 14.xlsx',
 'Jan 15.xlsx',
 'Jan 16.xlsx',
 'Jan 17.xlsx',
 'Jan 18.xlsx',
 'Jan 19.xlsx',
 'Jan 2.xlsx',
 'Jan 20.xlsx',
 'Jan 21.xlsx',
 'Jan 22.xlsx',
 'Jan 23.xlsx',
 'Jan 24.xlsx',
 'Jan 25.xlsx',
 'Jan 26.xlsx',
 'Jan 27.xlsx',
 'Jan 28.xlsx',
 'Jan 29.xlsx',
 'Jan 3.xlsx',
 'Jan 30.xlsx',
 'Jan 31.xlsx',
 'Jan 4.xlsx',
 'Jan 5.xlsx',
 'Jan 6.xlsx',
 'Jan 7.xlsx',
 'Jan 8.xlsx',
 'Jan 9.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [74]:
# Read in data from your city from January 1st.
jan_1 = pd.read_excel('data/Jan 1.xlsx', sheet_name='Atlanta')

In [6]:
jan_1

,prodcode,price_eu,weight_kg,quantity
0,4134,1.803357,7.107002,315.0
1,4011,1.906416,9.722703,429.0
2,4062,1.747032,2.245984,365.0
3,4159,1.827062,5.054962,287.0
4,4404,1.457843,5.406130,444.0
...,...,...,...,...
191,4404,0.830037,3.339435,373.0
192,4078,1.744919,4.904047,203.0
193,4412,0.965750,1.887790,387.0
194,4323,1.101243,1.298194,296.0


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [7]:
jan_1['price_usd'] = jan_1['price_eu']*1.1
jan_1['weight_lb'] = jan_1['weight_kg']*2.2

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [8]:
plu = pd.read_csv("plu-codes.csv")

In [9]:
jan_1 =jan_1.join(plu, lsuffix="_left", rsuffix="_right")

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [10]:
jan_1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product,plu_code
0,4134,1.803357,7.107002,315.0,1.983693,15.635404,Apple (Fuji),4131.0
1,4011,1.906416,9.722703,429.0,2.097058,21.389947,Apple (Gala),4134.0
2,4062,1.747032,2.245984,365.0,1.921736,4.941165,Apricot,3302.0
3,4159,1.827062,5.054962,287.0,2.009768,11.120917,Avocado,4225.0
4,4404,1.457843,5.406130,444.0,1.603627,11.893485,Banana,4011.0


In [11]:
jan_1.drop(['price_eu', 'weight_kg', 'plu_code'], axis =1, inplace =True)

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [12]:
jan_1['date']= 'Jan 1'

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [69]:
def process_data(file, city):
    df = pd.read_excel('data/' + file, sheet_name = city)
    df['price_usd'] = df['price_eu']*1.1
    df['weight_lb'] = df['weight_kg']*2.2
    df =df.join(plu, lsuffix="_left", rsuffix="_right")
    df.drop(['price_eu', 'weight_kg', 'plu_code'], axis =1, inplace =True)
    sp_2 = file.split('.')
    df['date'] = sp_2[0]
    df['city'] = city
    return df

Test your function out on a new file and city!

In [70]:
process_data('Jan 10.xlsx', 'Los Angeles')

,prodcode,quantity,price_usd,weight_lb,product,date,city
0,4272,374.0,3.215805,18.458957,Apple (Fuji),Jan 10,Los Angeles
1,4134,386.0,1.101397,4.144168,Apple (Gala),Jan 10,Los Angeles
2,4650,254.0,2.462905,16.272903,Apricot,Jan 10,Los Angeles
3,4309,220.0,1.402996,13.852859,Avocado,Jan 10,Los Angeles
4,4012,326.0,2.383897,10.723717,Banana,Jan 10,Los Angeles
...,...,...,...,...,...,...,...
194,4650,213.0,1.305891,5.691630,NaN,Jan 10,Los Angeles
195,4796,290.0,1.865070,19.355372,NaN,Jan 10,Los Angeles
196,4012,448.0,3.055497,7.577243,NaN,Jan 10,Los Angeles
197,4030,391.0,1.092079,19.929001,NaN,Jan 10,Los Angeles


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [15]:
city_df =[process_data('data/'+file,'Los Angeles') for file in files]

C:\Users\dmarm\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [16]:
city_df

[     prodcode  quantity  price_usd  weight_lb       product   date  \
 0        4134     315.0   1.983693  15.635404  Apple (Fuji)  Jan 1   
 1        4011     429.0   2.097058  21.389947  Apple (Gala)  Jan 1   
 2        4062     365.0   1.921736   4.941165       Apricot  Jan 1   
 3        4159     287.0   2.009768  11.120917       Avocado  Jan 1   
 4        4404     444.0   1.603627  11.893485        Banana  Jan 1   
 ..        ...       ...        ...        ...           ...    ...   
 191      4404     373.0   0.913040   7.346757           NaN  Jan 1   
 192      4078     203.0   1.919411  10.788904           NaN  Jan 1   
 193      4412     387.0   1.062325   4.153138           NaN  Jan 1   
 194      4323     296.0   1.211368   2.856027           NaN  Jan 1   
 195      4958     464.0   2.773385  14.741491           NaN  Jan 1   
 
             city  
 0    Los Angeles  
 1    Los Angeles  
 2    Los Angeles  
 3    Los Angeles  
 4    Los Angeles  
 ..           ...  
 191  

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [17]:
la_df = pd.concat(city_df)

In [18]:
la_df

,prodcode,quantity,price_usd,weight_lb,product,date,city
0,4134,315.0,1.983693,15.635404,Apple (Fuji),Jan 1,Los Angeles
1,4011,429.0,2.097058,21.389947,Apple (Gala),Jan 1,Los Angeles
2,4062,365.0,1.921736,4.941165,Apricot,Jan 1,Los Angeles
3,4159,287.0,2.009768,11.120917,Avocado,Jan 1,Los Angeles
4,4404,444.0,1.603627,11.893485,Banana,Jan 1,Los Angeles
...,...,...,...,...,...,...,...
123,4323,236.0,0.885784,19.235559,NaN,Jan 9,Los Angeles
124,4062,336.0,1.089004,19.423737,NaN,Jan 9,Los Angeles
125,4011,170.0,1.651145,17.931241,NaN,Jan 9,Los Angeles
126,4796,329.0,2.381774,14.297728,NaN,Jan 9,Los Angeles


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [19]:
city2_df =[process_data('data/'+file,'Atlanta') for file in files]

C:\Users\dmarm\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [20]:
city3_df =[process_data('data/'+file,'Austin') for file in files]

In [21]:
city4_df =[process_data('data/'+file,'Boston') for file in files]

In [22]:
city5_df =[process_data('data/'+file,'Chicago') for file in files]

In [23]:
city6_df =[process_data('data/'+file,'Denver') for file in files]

In [24]:
city7_df =[process_data('data/'+file,'New York') for file in files]

In [25]:
city8_df =[process_data('data/'+file,'San Francisco') for file in files]

In [26]:
city9_df =[process_data('data/'+file,'Seattle') for file in files]

In [27]:
city10_df =[process_data('data/'+file,'Washington, DC') for file in files]

In [28]:
atl_df = pd.concat(city2_df)

In [29]:
austin_df = pd.concat(city3_df)

In [30]:
bos_df = pd.concat(city4_df)

In [31]:
chi_df = pd.concat(city5_df)

In [32]:
den_df = pd.concat(city6_df)

In [33]:
ny_df = pd.concat(city7_df)

In [34]:
sf_df = pd.concat(city8_df)

In [35]:
sea_df = pd.concat(city9_df)

In [36]:
dc_df = pd.concat(city10_df)

In [37]:
la_df = la_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [38]:
atl_df = atl_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [39]:
austin_df = la_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [40]:
bos_df = bos_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [41]:
chi_df = chi_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [42]:
den_df = den_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [43]:
ny_df = ny_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [44]:
sf_df = sf_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [45]:
sea_df = sea_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [46]:
dc_df = dc_df.loc[:,['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]

In [47]:
dc_df.head()

,city,date,product,prodcode,quantity,weight_lb,price_usd
0,"Washington, DC",Jan 1,Apple (Fuji),4011,182.0,2.476043,1.578451
1,"Washington, DC",Jan 1,Apple (Gala),4404,363.0,3.097204,3.170313
2,"Washington, DC",Jan 1,Apricot,4272,309.0,10.484605,0.601788
3,"Washington, DC",Jan 1,Avocado,4131,354.0,14.948032,2.234031
4,"Washington, DC",Jan 1,Banana,4240,347.0,6.551917,1.824828


In [66]:
for file in files: 
    print(file)

Jan 1.xlsx
Jan 10.xlsx
Jan 11.xlsx
Jan 12.xlsx
Jan 13.xlsx
Jan 14.xlsx
Jan 15.xlsx
Jan 16.xlsx
Jan 17.xlsx
Jan 18.xlsx
Jan 19.xlsx
Jan 2.xlsx
Jan 20.xlsx
Jan 21.xlsx
Jan 22.xlsx
Jan 23.xlsx
Jan 24.xlsx
Jan 25.xlsx
Jan 26.xlsx
Jan 27.xlsx
Jan 28.xlsx
Jan 29.xlsx
Jan 3.xlsx
Jan 30.xlsx
Jan 31.xlsx
Jan 4.xlsx
Jan 5.xlsx
Jan 6.xlsx
Jan 7.xlsx
Jan 8.xlsx
Jan 9.xlsx


In [78]:
for city, file_out in city_dict.items():
    data_list = [process_data(file, city) for file in files]
    data_full = pd.concat(data_list, axis=0)
    data_full["city"] = city
    data_full = data_full.loc[:, 
        ["city", "date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]
    ]
    data_full.to_csv("/users/dmarm/downloads/excellab" + file_out, index=False) #worked on with John

In [75]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [51]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [83]:
At = pd.read_csv('/users/dmarm/downloads/excellabatl.csv')

In [84]:
Au = pd.read_csv('/users/dmarm/downloads/excellabatx.csv')

In [86]:
Bo = pd.read_csv('/users/dmarm/downloads/excellabbos.csv')

In [87]:
Ch = pd.read_csv('/users/dmarm/downloads/excellabchi.csv')

In [88]:
De = pd.read_csv('/users/dmarm/downloads/excellabden.csv')

In [89]:
La = pd.read_csv('/users/dmarm/downloads/excellablax.csv')

In [90]:
Ny = pd.read_csv('/users/dmarm/downloads/excellabnyc.csv')

In [91]:
Sf = pd.read_csv('/users/dmarm/downloads/excellabsf.csv')

In [92]:
Se = pd.read_csv('/users/dmarm/downloads/excellabsea.csv')

In [93]:
Dc = pd.read_csv('/users/dmarm/downloads/excellabdc.csv')

In [95]:
df_list =[]

In [97]:
df_list.append(At)

In [98]:
df_list.append(Au)

In [99]:
df_list.append(Bo)

In [100]:
df_list.append(Ch)

In [101]:
df_list.append(De)

In [102]:
df_list.append(La)

In [103]:
df_list.append(Ny)

In [104]:
df_list.append(Sf)

In [105]:
df_list.append(Se)

In [106]:
df_list.append(Dc)

In [107]:
df = pd.concat(df_list)

In [110]:
df.groupby('city')['quantity'].mean()

city
Atlanta           302.485306
Austin            301.048298
Boston            298.806536
Chicago           301.686186
Denver            301.012745
Los Angeles       301.531276
New York          299.482863
San Francisco     298.979895
Seattle           300.333694
Washington, DC    300.576421
Name: quantity, dtype: float64

In [111]:
df.groupby('city')['weight_lb'].mean()

city
Atlanta           11.929027
Austin            12.092775
Boston            12.063057
Chicago           12.101555
Denver            12.129850
Los Angeles       12.167527
New York          12.090294
San Francisco     12.237399
Seattle           11.925486
Washington, DC    11.930747
Name: weight_lb, dtype: float64

In [112]:
df.groupby('city')['price_usd'].mean()

city
Atlanta           1.937980
Austin            1.937456
Boston            1.900973
Chicago           1.930026
Denver            1.932088
Los Angeles       1.918331
New York          1.912662
San Francisco     1.925830
Seattle           1.903315
Washington, DC    1.943107
Name: price_usd, dtype: float64

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).